# Website A/B Testing - Lab

## Introduction

In this lab, you'll get another chance to practice your skills at conducting a full A/B test analysis. It will also be a chance to practice your data exploration and processing skills! The scenario you'll be investigating is data collected from the homepage of a music app page for audacity.

## Objectives

You will be able to:
* Analyze the data from a website A/B test to draw relevant conclusions
* Explore and analyze web action data

## Exploratory Analysis

Start by loading in the dataset stored in the file 'homepage_actions.csv'. Then conduct an exploratory analysis to get familiar with the data.

> Hints:
    * Start investigating the id column:
        * How many viewers also clicked?
        * Are there any anomalies with the data; did anyone click who didn't view?
        * Is there any overlap between the control and experiment groups? 
            * If so, how do you plan to account for this in your experimental design?

In [1]:
#Your code here
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns
from statsmodels.stats.power import TTestIndPower

In [28]:
# Import and view the data
df = pd.read_csv('homepage_actions.csv')
df.head()

,timestamp,id,group,action
0,2016-09-24 17:42:27.839496,804196,experiment,view
1,2016-09-24 19:19:03.542569,434745,experiment,view
2,2016-09-24 19:36:00.944135,507599,experiment,view
3,2016-09-24 19:59:02.646620,671993,control,view
4,2016-09-24 20:26:14.466886,536734,experiment,view


In [29]:
# Experiment v Control Groups
df.group.value_counts()

control       4264
experiment    3924
Name: group, dtype: int64

In [30]:
# View v Clicks
df.action.value_counts()

view     6328
click    1860
Name: action, dtype: int64

In [31]:
df[df.group=='control'].pivot(index='id', columns='action')

timestamp                                group  \
action                       click                        view    click   
id                                                                        
182994  2016-12-14 13:20:29.564080  2016-12-14 13:19:00.940677  control   
183089                         NaN  2016-09-29 23:04:42.722325      NaN   
183248  2016-11-24 02:31:00.749999  2016-11-24 02:30:11.939588  control   
183515                         NaN  2016-11-03 05:00:34.258889      NaN   
183524                         NaN  2016-10-03 18:55:46.601376      NaN   
...                            ...                         ...      ...   
936786                         NaN  2016-12-06 14:06:03.807870      NaN   
937003                         NaN  2016-10-02 22:23:52.952457      NaN   
937073                         NaN  2016-12-20 13:05:00.579131      NaN   
937108                         NaN  2017-01-07 11:50:17.313666      NaN   
937217  2016-11-14 08:01:25.774204  2016-11-14 08:00:01.730193  control   

                 
action     view  
id               
182994  control  
183089  control  
183248  control  
183515  control  
183524  control  
...         ...  
936786  control  
937003  control  
937073  control  
937108  control  
937217  control  

[3332 rows x 4 columns]

## Conduct a Statistical Test

Conduct a statistical test to determine whether the experimental homepage was more effective than that of the control group.

In [32]:
#Your code here
# Create a variable records
df['records'] = 1.0

In [33]:
# Using the records value, group the dataframe by control or experiment
control_df = df[df['group'] == 'control']
experiment_df = df[df['group'] == 'experiment']

# Grouping the control and experiment group
control = control_df.pivot('id', 'action', 'records')
control_group = control.fillna(value=0)

experiment = experiment_df.pivot('id', 'action', 'records')
experiment_group = experiment.fillna(value=0)

## Verifying Results

One sensible formulation of the data to answer the hypothesis test above would be to create a binary variable representing each individual in the experiment and control group. This binary variable would represent whether or not that individual clicked on the homepage; 1 for they did and 0 if they did not. 

The variance for the number of successes in a sample of a binomial variable with n observations is given by:

## $n\bullet p (1-p)$

Given this, perform 3 steps to verify the results of your statistical test:
1. Calculate the expected number of clicks for the experiment group, if it had the same click-through rate as that of the control group. 
2. Calculate the number of standard deviations that the actual number of clicks was from this estimate. 
3. Finally, calculate a p-value using the normal distribution based on this z-score.

### Step 1:
Calculate the expected number of clicks for the experiment group, if it had the same click-through rate as that of the control group. 

In [36]:
#Your code here
control_group_rate = control_group['click'].mean()
expected_no_of_clicks = control_group_rate * len(experiment_group)
print(expected_no_of_clicks)

838.0168067226891


### Step 2:
Calculate the number of standard deviations that the actual number of clicks was from this estimate.

In [39]:
#Your code here

# The standard deviations
# formula = n * p * (1-p)
actual = len(experiment_group) * control_group_rate * (1 - control_group_rate)
std = np.sqrt(actual)

# The X-BAR
actual_clicks = experiment_group['click'].sum()

# The z-score
# z = (X - MEAN)/STD
z  = (actual_clicks - expected_no_of_clicks)/ std
z

3.6625360854823588

### Step 3: 
Finally, calculate a p-value using the normal distribution based on this z-score.

In [40]:
#Your code here
import scipy.stats as stats
p_val = stats.norm.sf(z)
print(p_val)

0.00012486528006951198


### Analysis:

Does this result roughly match that of the previous statistical test?

> Comment: **Your analysis here**

We would reject the null hypothesis in favor the alternate hypothesis.

## Summary

In this lab, you continued to get more practice designing and conducting AB tests. This required additional work preprocessing and formulating the initial problem in a suitable manner. Additionally, you also saw how to verify results, strengthening your knowledge of binomial variables, and reviewing initial statistical concepts of the central limit theorem, standard deviation, z-scores, and their accompanying p-values.